In [56]:
import pandas as pd
from nltk.stem import PorterStemmer
import time
import spacy
# from sentence_similarity import sentence_similarity
from math import sqrt

In [2]:
ps = PorterStemmer() # stemming for better results 

zehrs_regular_index = {}
zehrs_flyer_index = {}

regular_priced = pd.read_csv(f'clean_data/{"zehrs"}/regular_prices.csv')
flyer = pd.read_csv(f'clean_data/{"zehrs"}/flyer_deals.csv')


for index, row in regular_priced.iterrows():
    product_name = row['product']
#     brand = row['brand'] # consider adding brand into index if we want customers to be brand specific 
    
    product = ps.stem(product_name).lower()
    
    for word in product.split(): 
        if word not in zehrs_regular_index.keys():
            zehrs_regular_index[word] = [index]
        else:
            zehrs_regular_index[word].append(index)
            
for index, row in flyer.iterrows():
    product_name = row['product_name']
#     brand = row['brand'] # consider adding brand into index if we want customers to be brand specific 
    try:
        product = ps.stem(product_name).lower()
#         print(product)
        for word in product.split(): 
            if word not in zehrs_flyer_index.keys():
                zehrs_flyer_index[word] = [index]
            else:
                zehrs_flyer_index[word].append(index)
    except:
        continue

In [3]:
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

In [9]:
grocery_list = ['2% milk', 'Cheddar Cheese', 'white sliced bread', 'ground beef', 
                'clementines', 'chicken breast', 'potatos']


zehrs_results = pd.DataFrame()

store = 'zehrs'

final_selection = pd.DataFrame(columns = ['list_item', 'store', 'product_name', 'price', 'per_unit_price', 'source'])

for item in grocery_list:
    item_selection = pd.DataFrame(columns = ['list_item', 'store', 'product_name', 'price', 'per_unit_price', 'source'])
    
    stem_item = ps.stem(item).lower()
#     print(item)
    idxs = []
    
    for word in stem_item.split():
        word_idxs = zehrs_regular_index[word]
        idxs.extend(word_idxs)
        
    df = regular_priced.iloc[idxs]
    
#     print(df.head())
    
    for index, row in df.iterrows():
        product_name = row['product']
        per_unit_price = row['per_unit_price2']
        price = row['price2']
        
        similarity = jaccard_similarity(stem_item.split(' '), ps.stem(product_name).lower().split(' '))

        if similarity >= 0.5: # can tweak threshold but this is a good one for now  
            data = { 'list_item':item, 'store':store, 'product_name':product_name, 'price':price, 'per_unit_price':per_unit_price, 'similarity':similarity, 'source': 'reg' }
            item_selection = item_selection.append(data, ignore_index=True)
            
    cheapest_item = item_selection.sort_values(by=['per_unit_price', 'similarity'], ascending = [True, False])

    final_selection = final_selection.append(dict(cheapest_item.iloc[0]), ignore_index=True)

In [11]:
final_selection

,list_item,store,product_name,price,per_unit_price,source,similarity
0,2% milk,zehrs,2% Milk,4.99,0.00250,reg,1.000000
1,Cheddar Cheese,zehrs,"Beef Burger, Cheddar Cheese",14.99,0.01330,reg,0.500000
2,white sliced bread,zehrs,White Bread,2.69,0.00400,reg,0.666667
3,ground beef,zehrs,Lean Ground Beef,10.00,0.01100,reg,0.666667
4,clementines,zehrs,Organic Clementines,9.99,0.01100,reg,0.500000
5,chicken breast,zehrs,"Chicken Breast Fillets, Boneless",8.72,0.01982,reg,0.500000
6,potatos,zehrs,Russet Potatoes,2.99,0.00070,reg,0.500000


## Putting it all together

In [39]:
# build index for each store 

ps = PorterStemmer() # stemming for better results 

stores = ['zehrs', 'no_frills', 'valu_mart']


# make variables 
for store in stores:
    globals()[f"{store}_regular_index"] = {} # dynamically create variable names 
    globals()[f"{store}_flyer_index"] = {}

for store in stores:

    regular_priced = pd.read_csv(f'clean_data/{store}/regular_prices.csv')
    flyer = pd.read_csv(f'clean_data/{store}/flyer_deals.csv')


    for index, row in regular_priced.iterrows():
        product_name = row['product']

        product = ps.stem(product_name).lower()

        for word in product.split(): 
            if word not in globals()[f"{store}_regular_index"].keys():
                globals()[f"{store}_regular_index"][word] = [index]
            else:
                globals()[f"{store}_regular_index"][word].append(index)

    for index, row in flyer.iterrows():
        product_name = row['product_name']
        
        try: # in case of numbers only in word 
            product = ps.stem(product_name).lower()

            for word in product.split(): 
                if word not in globals()[f"{store}_flyer_index"].keys():
                    globals()[f"{store}_flyer_index"][word] = [index]
                else:
                    globals()[f"{store}_flyer_index"][word].append(index)
        except:
            continue

In [57]:
def squared_sum(x):
    """ return 3 rounded square rooted value """
    return round(sqrt(sum([a*a for a in x])),3)


def cos_similarity(x,y):
    """ return cosine similarity between two lists """
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = squared_sum(x)*squared_sum(y)
    return round(numerator/float(denominator),3)


In [127]:
# search and find 

start_time = time.time()

# model= sentence_similarity(model_name='distilbert-base-uncased',embedding_type='sentence_embedding')
nlp = spacy.load('en_core_web_sm')

print('loaded sim model in', time.time() - start_time, 'seconds')

ps = PorterStemmer() # stemming for better results 

grocery_list = ['2% milk', 'Cheddar Cheese', 'white sliced bread', 'ground beef', 
                'clementines', 'potatoes', 'chicken breast']


stores = ['no_frills', 'valu_mart', 'zehrs']


# make variables 
for store in stores:
    globals()[f"{store}_results"] = pd.DataFrame() # dynamically create variable names 

    
for store in stores:
    regular_priced = pd.read_csv(f'clean_data/{store}/regular_prices.csv')
    flyer = pd.read_csv(f'clean_data/{store}/flyer_deals.csv')
    
    final_selection = pd.DataFrame(columns = ['list_item', 'store', 'product_name', 'price', 'per_unit_price', 'source'])
    
    for item in grocery_list:
        item_selection = pd.DataFrame(columns = ['list_item', 'store', 'product_name', 'price', 'per_unit_price', 'source'])

        stem_item = ps.stem(item).lower()
        reg_idxs = []
        flyer_idxs = []

        # get all relevant indexes for regular priced data
        for word in stem_item.split():
            try: # list item word not in index
                word_idxs = globals()[f"{store}_regular_index"][word]
                reg_idxs.extend(word_idxs)
            except: continue

        if not reg_idxs: # no indicies returned
            reg_df = pd.DataFrame() # no results: return empty df
        else:
            reg_df = regular_priced.iloc[reg_idxs]
        
        # get all relevant indexes for sale priced data
        for word in stem_item.split():
            try: # list item word not in index
                word_idxs = globals()[f"{store}_flyer_index"][word]
                flyer_idxs.extend(word_idxs)
            except: continue

        if not flyer_idxs: # no indicies returned
            flyer_df = pd.DataFrame() # no results: return empty df
        else:
            flyer_df = flyer.iloc[flyer_idxs]


#         print(len(reg_df))
#         print(len(flyer_df))

        for index, row in reg_df.iterrows():
            product_name = row['product'].split(',')[0]
            per_unit_price = row['per_unit_price2']
            price = row['price2']

            similarity = jaccard_similarity(stem_item.split(' '), ps.stem(product_name).lower().split(' '))

#             similarity = model.get_score(stem_item.lower(), ps.stem(product_name).lower(),metric="cosine")

#             strings = [ps.stem(product_name).lower(), stem_item.lower()]
#             embeddings = [nlp(string).vector for string in strings]
#             similarity = cos_similarity(embeddings[0], embeddings[1])
    
            if similarity >= 0.5: # can tweak threshold but this is a good one for now  
                data = { 'list_item':item, 'store':store, 'product_name':product_name, 'price':price, 'per_unit_price':per_unit_price, 'similarity':similarity, 'source': 'reg' }
                item_selection = item_selection.append(data, ignore_index=True)
                
                
        ##### search flyer data #####
        for index, row in flyer_df.iterrows():

            try: 
                product_name = row['product_name'].replace(',', '')
                price = row['price2']
                per_unit_price = row.per_unit_price2

                # find items  
                similarity = jaccard_similarity(stem_item.split(' '), ps.stem(product_name).lower().split(' '))

                if store == 'zehrs' and item == 'chicken breast': print(product_name, similarity)
#                 similarity = model.get_score(stem_item.lower(), ps.stem(product_name).lower(),metric="cosine")

#                 strings = [ps.stem(product_name).lower(), stem_item.lower()]
#                 embeddings = [nlp(string).vector for string in strings]
#                 similarity = cos_similarity(embeddings[0], embeddings[1])
                    
                if similarity >= 0.1: # can tweak threshold but this is a good one for now  
                    data = { 'list_item':item, 'store':store, 'product_name':product_name, 'price':price, 'per_unit_price': per_unit_price, 'similarity':similarity, 'source': 'flyer' }
                    item_selection = item_selection.append(data, ignore_index=True)
               
            except: continue

        try:
            # find lowest price from top similarities
            
            # ************** need to decide between per unit pricing and total price difference
            # ************** maybe some units need per unit some dont 
            cheapest_item = item_selection.sort_values(by=['per_unit_price', 'source', 'similarity'], ascending = [True, True, False])
#             cheapest_item = item_selection.sort_values(by=['price'])

            final_selection = final_selection.append(dict(cheapest_item.iloc[0]), ignore_index=True)
        except: continue
            
    globals()[f"{store}_results"] = final_selection
    
print(time.time() - start_time, 'seconds')

loaded sim model in 0.6016807556152344 seconds
CHICKEN BREASTS BONE-IN SKIN-ON OR SPLIT CHICKEN WINGS 0.125
CHICKEN BREASTS BONE-IN SKIN-ON OR SPLIT CHICKEN WINGS 0.125
CHICKEN OR PULLED PORK TACO TRAYS 975 G-1.1 KG 0.1
SUFRA® HALAL CHICKEN DRUMSTICKS OR THIGHS 840 g 0.1111111111111111
MAPLE LODGE FARMS ULTIMATE CHICKEN SAUSAGES 500 g 0.1111111111111111
HOUSTON FLATTENED PORTUGUESE WHOLE CHICKEN 0.16666666666666666
MAPLE LODGE FARMS ULTIMATE CHICKEN SAUSAGES 500 G 0.1111111111111111
1.6462018489837646 seconds


# outputs

In [128]:
zehrs_results

,list_item,store,product_name,price,per_unit_price,source,similarity
0,2% milk,zehrs,2% Milk,4.99,0.00250,reg,1.000000
1,Cheddar Cheese,zehrs,Medium Cheddar Cheese,9.79,0.01400,reg,0.666667
2,white sliced bread,zehrs,White Bread,2.69,0.00400,reg,0.666667
3,ground beef,zehrs,Lean Ground Beef,10.00,0.01100,reg,0.666667
4,clementines,zehrs,Organic Clementines,9.99,0.01100,reg,0.500000
5,potatoes,zehrs,Russet Potatoes,2.99,0.00070,reg,0.500000
6,chicken breast,zehrs,Chicken Breast,9.36,0.00659,reg,1.000000


In [129]:
no_frills_results

,list_item,store,product_name,price,per_unit_price,source,similarity
0,2% milk,no_frills,2% Milk,4.99,0.00250,reg,1.000000
1,Cheddar Cheese,no_frills,Medium Cheddar Cheese,4.99,0.01250,reg,0.666667
2,white sliced bread,no_frills,White Bread,1.79,0.00270,reg,0.666667
3,ground beef,no_frills,Medium Ground Beef,14.26,0.01100,reg,0.666667
4,clementines,no_frills,Clementines,3.99,0.00170,reg,1.000000
5,potatoes,no_frills,Sweet Potato,1.37,0.00218,reg,0.500000
6,chicken breast,no_frills,Chicken Breast Strips,10.99,0.01370,reg,0.666667


In [130]:
valu_mart_results

,list_item,store,product_name,price,per_unit_price,source,similarity
0,2% milk,valu_mart,2% Milk,4.99,0.00250,reg,1.000000
1,Cheddar Cheese,valu_mart,Old Cheddar Cheese,9.79,0.01400,reg,0.666667
2,white sliced bread,valu_mart,White Bread,2.79,0.00410,reg,0.666667
3,ground beef,valu_mart,Beef Ground Medium,11.26,0.00880,reg,0.666667
4,clementines,valu_mart,Clementines,6.99,0.00300,reg,1.000000
5,potatoes,valu_mart,Russet Potatoes,3.99,0.00090,reg,0.500000
6,chicken breast,valu_mart,Free From Chicken Breast,14.00,0.01321,reg,0.500000


In [131]:
zehrs_per_unit_subtotal = round(zehrs_results.per_unit_price.sum(),3)
no_frills_per_unit_subtotal = round(no_frills_results.per_unit_price.sum(),3)
valu_mart_per_unit_subtotal = round(valu_mart_results.per_unit_price.sum(),3)

zehrs_subtotal = round(zehrs_results.price.sum(),2)
no_frills_subtotal = round(no_frills_results.price.sum(),2)
valu_mart_subtotal = round(valu_mart_results.price.sum(),2)

lowest_price = min(zehrs_per_unit_subtotal, no_frills_per_unit_subtotal, valu_mart_per_unit_subtotal)

print(lowest_price)
print(f'Zehrs: {zehrs_per_unit_subtotal}, {zehrs_subtotal}')
print(f'No Frills: {no_frills_per_unit_subtotal}, {no_frills_subtotal}')
print(f'Valu Mart: {valu_mart_per_unit_subtotal}, {valu_mart_subtotal}')

0.046
Zehrs: 0.05, 49.81
No Frills: 0.046, 42.38
Valu Mart: 0.047, 53.81


In [132]:
item_selection

,list_item,store,product_name,price,per_unit_price,source,similarity
0,chicken breast,zehrs,Chicken Breast,12.00,12.000000,reg,1.000000
1,chicken breast,zehrs,Free From Chicken Breast,27.32,0.025770,reg,0.500000
2,chicken breast,zehrs,Boneless Skinless Chicken Breasts,15.00,15.000000,reg,0.500000
3,chicken breast,zehrs,Free From Chicken Breast,14.09,0.027090,reg,0.500000
4,chicken breast,zehrs,Air Chilled Chicken Breasts,23.13,0.023130,reg,0.500000
5,chicken breast,zehrs,Chicken Breast Fillets,8.72,0.019820,reg,0.666667
6,chicken breast,zehrs,Organics Split Chicken Breasts,14.94,0.025330,reg,0.500000
7,chicken breast,zehrs,Chicken Breast Fillet,12.29,0.019820,reg,0.666667
8,chicken breast,zehrs,Chicken Breast,9.36,0.006590,reg,1.000000
9,chicken breast,zehrs,Free From Chicken Breasts,12.97,0.020260,reg,0.500000


In [133]:
item_selection.sort_values(by=['per_unit_price', 'source', 'similarity'], ascending = [True, True, False])

,list_item,store,product_name,price,per_unit_price,source,similarity
8,chicken breast,zehrs,Chicken Breast,9.36,0.006590,reg,1.000000
13,chicken breast,zehrs,CHICKEN BREASTS BONE-IN SKIN-ON OR SPLIT CHICK...,2.99,0.006592,flyer,0.125000
14,chicken breast,zehrs,CHICKEN BREASTS BONE-IN SKIN-ON OR SPLIT CHICK...,2.99,0.006592,flyer,0.125000
16,chicken breast,zehrs,SUFRA® HALAL CHICKEN DRUMSTICKS OR THIGHS 840 g,8.00,0.008000,flyer,0.111111
17,chicken breast,zehrs,MAPLE LODGE FARMS ULTIMATE CHICKEN SAUSAGES 500 g,5.99,0.011980,flyer,0.111111
18,chicken breast,zehrs,HOUSTON FLATTENED PORTUGUESE WHOLE CHICKEN,5.49,0.012103,flyer,0.166667
5,chicken breast,zehrs,Chicken Breast Fillets,8.72,0.019820,reg,0.666667
7,chicken breast,zehrs,Chicken Breast Fillet,12.29,0.019820,reg,0.666667
10,chicken breast,zehrs,Chicken Breast Fillets,8.72,0.019820,reg,0.666667
12,chicken breast,zehrs,Chicken Breast Fillet,12.29,0.019820,reg,0.666667
